In [1]:
from scraper import MangaScraper
from image_processory import OCR
from PIL import Image
import gradio as gr
import os

uploaded_images = []

# Dummy function for translation (replace with actual translation logic)
def translate_manga(language):
    print("Translating manga to", language)
    global uploaded_images
    ocr = OCR()
    
    translated_images = []
    # Iterate over images and yield the translated image one by one
    for uploaded_image in uploaded_images:
        translated_image = ocr.translate_image(uploaded_image, language=language)
        translated_images.append(translated_image)
        # Use gr.update to progressively update the gallery
        yield gr.update(value=translated_images)

def process_uploaded_files(url):
    global uploaded_images
    uploaded_images = []
    manga_scraper = MangaScraper()
    
    scraped_images_path = manga_scraper.scrape_manga_pill(url)
    if scraped_images_path != False:
        print("Scraping Completed")
        images = [Image.open(file) for file in scraped_images_path]
        uploaded_images = scraped_images_path
    else:
        print("An error occurred while scraping the manga")
        images = []

    return images

def process_direct_upload(files):
    global uploaded_images
    print(type(files),type(files[0]),files)
    uploaded_images = [file[0] for file in files]
    return [Image.open(name) for name in uploaded_images]

# Gradio interface
with gr.Blocks() as manga_translation_ui:
    languages = ["Hindi", "Kannada", "Bengali", "Gujarati"]
    # endpoints = ["Groq", "Sarvam"]
    gr.Markdown("## Manga Translation Program")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Manga Pages")
            original_images = gr.Gallery(label="Upload Manga Pages", interactive=True, height="900px")
        
        with gr.Column():
            gr.Markdown("### Translated Manga Pages")
            translated_images = gr.Gallery(label="Translated Manga Pages", interactive=False, height="900px")
    
    url_input = gr.Textbox(label="Enter Manga URL here...", placeholder="Enter Manga URL here...", lines=1)
    language_dropdown = gr.Dropdown(label="Select a language", choices=languages, value=languages[0])
    # endpoint_dropdown = gr.Dropdown(label="Select an endpoint", choices=endpoints, value=endpoints[0])

    upload_btn = gr.Button("Upload Manga Pages")
    translate_btn = gr.Button("Translate")
   
    upload_btn.click(process_uploaded_files, inputs=[url_input], outputs=[original_images])
    
    # No need for `stream=True`, use generator for progressively updating output

    translate_btn.click(fn=translate_manga, inputs=[language_dropdown], outputs=[translated_images])
    original_images.upload(process_direct_upload, inputs=[original_images], outputs=[original_images])

# Launch the Gradio app
manga_translation_ui.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
